   # ESAME

In [86]:
x1 <- c(1,1,0,1,0,0,1,0,1,0)
x2 <- c(0,1,0,1,0,1,1,0,1,0)
x3 <- c(0,1,1,1,0,1,1,0,1,0)


df = data.frame(x1,x2,x3)
df


x1,x2,x3
1,0,0
1,1,1
0,0,1
1,1,1
0,0,0
0,1,1
1,1,1
0,0,0
1,1,1
0,0,0


In [87]:
n <- ncol(df)
u <- 2

r <- as.numeric(lengths(lapply(df, unique))) #numero di valori unici in ogni nodo
m <- as.numeric(lapply(df, length))  #numero di valori di ogni nodo
r.vec <- lapply(df, unique)  #quali sono i valori unici per ogni nodo


In [88]:
uq_elem=list()

for(i in 1:n)
{
      uq_elem[[i]] <-  unlist(r.vec[i], use.names=F) 

}

### funzione Nijk


In [89]:
## calcolo N ijk
N <- function(df, i, j, k, prec, u) {
    
    
    if(j==0){ 
        count <- length(which(df[,c(i)]==uq_elem[[i]][k]))
            }
   
    if( (length(prec)==1)  &  (j!=0)){ # because for (a in c(0:-1)) would do it twice instead of never
        
                                                       
        count <- length(which(  (df[,c(i)]==uq_elem[[i]][k]) & (df[,c(prec)]==uq_elem[[prec]][j])  ))
        

        }
    
        
    if((length(prec)>=2) &  (j!=0)){
        
        size <- length(prec)
        
        indices <- numeric(size)
        
        x <- j-1
        
        for (a in c(0:(size-2)) ){ # in R for loop extreme values are included 
            indices[size-a]<-(x%%r[prec[size-a]])+1 # the indices from last to second are given by the remainder of the division +1
            x<-(x%/%r[prec[size-a]])
        }
        indices[1] <- x+1 # the first index is given by the result of the last division +1
        

        
        l0 <- which( (df[,c(i)]==uq_elem[[i]][k]) ) 
        h <- 1
        l <- list()

        for (node in prec){

            l[[h]] <- which( (df[,c(node)]==uq_elem[[node]][indices[h]]) )  

            h <- h+1
         }
        l[[length(l)+1]] <- l0
        
        count <- length(Reduce(intersect, l)) 
        }
    
    
    return (count)
      
}

### funzione Nij

In [90]:

#calcolo N ij

N.sum <-function(df, i, j, prec, r, u) {
    
    c <- 0
    
    for (k in c(1:r[i])) {
         
        
         c <- c + N(df, i, j, k, prec, u)
        
        }
    
    return ( c )
    
    }


### funzione g



In [91]:
product <-function(df, i, j, r, prec, u) {
    
    prod <- 1
    
    for (k in c(1:r[i])){
            
            prod<-prod*factorial(N(df, i, j, k, prec, u))
            
        }
    
    return(prod)
    }

In [92]:
g <-function(df, i, prec, r, u) {
    
    if (prec[1]==0){

        p <-  factorial(r[i]-1)/factorial(N.sum(df, i, 0, prec, r, u)+r[i]-1)*product(df, i, 0, r, prec, u)
            
    }
    
    if ((length(prec)==1)  &  (prec[1]!=0)){


        
        p <- 1
        
        for (j in c(1:r[prec[1]])){
            
            p<- p*factorial(r[i]-1)/factorial(N.sum(df, i, j, prec, r, u)+r[i]-1)*product(df, i, j, r, prec, u)
            
        }
        
    }
    
    if((length(prec)>=2) &  (prec[1]!=0)){

        size <- length(prec)
        
        
        p <- 1
        
        for (j in c(1:prod(r[prec])) ){  # that function will do the work
        
            
            p<- p*factorial(r[i]-1)/factorial(N.sum(df, i, j, prec, r, u)+r[i]-1)*product(df, i, j, r, prec, u)
           
        }
        
    }
    return (p)
    }

## K2

### powerset function

In [93]:
powerset <- function(set, u) { 
    set<- set[set>0]
    
    len <- length(set)
    masks <- 2^(1:len-1)
    pow <-lapply( 1:2^len-1, function(x) set[ bitwAnd(x, masks) != 0 ] )
                 
    array<-list()         
    for (i in c(1:length(pow))){
        if ( (length(pow[[i]])>0) & (length(pow[[i]])<=u) ){
            array[i] <- pow[i]
        }
       
        
    }
                 
    array <- array[-which(sapply(array, is.null))]
    return(array)
}
                 



In [94]:
for (i in c(1:n)){
    
    prec <- 0
    P.old <- g(df, i, prec, r, u)
    
    OKtoProceed <- T
    
    if (i==1){
        OKtoProceed <- F    #se non ho nodi predecessori non entro nel ciclo
        }

    while ((OKtoProceed)){
        for (prec in powerset( c(1:(i-1))[], u )){
            
            P.new <- g(df, i, prec, r, u)
            
            if (P.new > P.old){
                P.old <- P.new    
                x <- prec

            }
        }   
        cat('node', x ,' parent of node', i)
            
        OKtoProceed <- F
        
    }
    cat('\n')
    
}


node 1  parent of node 2
node 2  parent of node 3
